In [ ]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

# Load dataset
df = pd.read_csv('rental_info.csv')

# Convert date columns to datetime
df["return_date"] = pd.to_datetime(df["return_date"])
df['rental_date'] = pd.to_datetime(df["rental_date"])

# Compute rental duration
df['rental_length'] = df['return_date'] - df['rental_date']
df['rental_length_days'] = df['rental_length'].dt.days  # target variable

# Create dummy variables from special_features
df["deleted_scenes"] = np.where(df["special_features"].str.contains("Deleted Scenes"), 1, 0)
df["behind_the_scenes"] = np.where(df["special_features"].str.contains("Behind the Scenes"), 1, 0)

# Select features and remove leakage columns
X = df.drop(columns=[
    "rental_length_days",  # target
    "rental_length",        # derived from target
    "rental_date",           # leaks target info
    "return_date",           # leaks target info
    "special_features"       # text already encoded
])

# Target variable
y = df["rental_length_days"]

# Remove non-numeric columns (Lasso needs numeric data)
non_numeric_cols = X.select_dtypes(include=['object']).columns
X = X.drop(columns=non_numeric_cols)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=9
)

# Scale features (important for Lasso)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Lasso model for feature selection
lasso = Lasso(alpha=0.01, random_state=9)
lasso.fit(X_train_scaled, y_train)

# Get feature coefficients
lasso_coef = pd.Series(lasso.coef_, index=X_train.columns)
print(lasso_coef)

# Select important features (non-zero coefficients)
selected_cols = lasso_coef[lasso_coef != 0].index
print("Selected features:", selected_cols)

# Subset dataset with selected features
X_train_selected = X_train[selected_cols]
X_test_selected = X_test[selected_cols]

# Hyperparameter tuning using GridSearchCV
from sklearn.model_selection import GridSearchCV

lasso = Lasso(random_state=9)

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10]
}

grid = GridSearchCV(lasso, param_grid, cv=5, scoring="neg_mean_squared_error")
grid.fit(X_train_scaled, y_train)

# Best alpha value
print("Best alpha:", grid.best_params_)

# Best tuned Lasso model
best_model = grid.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test_scaled)

# Compute Mean Squared Error
best_mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", best_mse)